In [23]:
# Importing required libraries 
from keras.models import Sequential, Model, model_from_json
import matplotlib.pyplot as plt
import keras 
import pickle
import wave  # !pip install wave
import os
import pandas as pd
import numpy as np
import sys
import warnings
import librosa
import librosa.display
import IPython.display as ipd  # To play sound in the notebook

In [24]:
data, sampling_rate = librosa.load('chunk_85-90s.wav')
#ipd.Audio('chunk_85-90s.wav')

In [18]:
# loading json and model architecture 
json_file = open('model_json.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("saved_models/Emotion_Model.h5")
print("Model Loaded")

# the optimiser
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

Model Loaded


In [19]:
# Lets transform the dataset so we can apply the predictions
X, sample_rate = librosa.load('chunk_85-90s.wav'
                              ,res_type='kaiser_fast'
                              ,duration=2.5
                              ,sr=44100
                              ,offset=0.5
                             )

sample_rate = np.array(sample_rate)
mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
newdf = pd.DataFrame(data=mfccs).T
newdf

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,-11.294938,-15.125132,-24.727261,-22.900198,-23.044731,-18.764437,-18.688389,-22.94228,-25.084158,-26.657005,...,-31.310795,-31.060823,-30.474926,-32.967495,-38.339394,-30.60619,-23.890635,-19.444794,-16.80761,-13.034663


In [20]:
# Apply predictions
newdf= np.expand_dims(newdf, axis=2)
newpred = loaded_model.predict(newdf, 
                         batch_size=16, 
                         verbose=1)

newpred

1/1 [==============================] - 0s 114ms/step


array([[5.2150174e-20, 9.2852874e-13, 1.0740721e-19, 3.7681625e-20,
        1.7721507e-18, 5.8185166e-01, 8.0701357e-06, 2.6322040e-30,
        9.4541363e-12, 1.5892900e-12, 7.4648502e-25, 1.9169160e-14,
        4.1814026e-01, 3.3527234e-14]], dtype=float32)

In [22]:
# above output in numbers, map above output with saved labels(last notebook)

filename = 'labels'
infile = open(filename,'rb')
lb = pickle.load(infile)
infile.close()

# Get the final predicted label
final = newpred.argmax(axis=1)
final = final.astype(int).flatten()
final = (lb.inverse_transform((final)))
print(final) #emotion(final) #gender(final)

['female_sad']
